In [191]:
import pandas as pd
import os
import re
import numpy as np
import nibabel
import argparse
import textwrap
import time
from multiprocessing import Pool
from numpy import count_nonzero
import statsmodels
import itertools

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from scipy import stats

%matplotlib inline

In [192]:
BL_thal = pd.read_csv('/Users/minji/Downloads/BL_thalamus.csv', index_col=0)
FU_thal = pd.read_csv('/Users/minji/Downloads/FU_thalamus.csv', index_col=0)

In [193]:
BL_thal_subj=BL_thal['subject'].drop_duplicates()

In [194]:
FU_thal_subj=FU_thal['subject'].drop_duplicates()

In [195]:
BL_thal_subj.shape

(52,)

In [196]:
FU_thal_subj.shape

(42,)

In [197]:
[x for x in FU_thal_subj if x not in BL_thal_subj.unique()]

['UMO27_LCY']

In [198]:
[x for x in BL_thal_subj if x not in FU_thal_subj.unique()]

['NOR22_KJH',
 'NOR15_KKY',
 'NOR05_PSM',
 'NOR21_JHK',
 'UMO25_CEG',
 'DNO10_YBI',
 'UMO31_CJB',
 'DNO24_KYH',
 'UMO32_SMJ',
 'UMO30_HWK',
 'DNO12_KYJ']

In [199]:
FU_thal_subj_list=FU_thal_subj.tolist()

In [200]:
BL_thal_wFU=BL_thal[BL_thal['subject'].isin(FU_thal_subj_list)]

In [201]:
BL_thal.shape

(4160, 13)

In [202]:
BL_thal_wFU.shape

(3280, 13)

In [203]:
len(BL_thal_wFU['subject'].drop_duplicates())

41

In [204]:
BL_wFU_thal_subj=BL_thal_wFU['subject'].drop_duplicates()

In [205]:
BL_wFU_thal_subj_list=BL_wFU_thal_subj.tolist()

In [206]:
len([x for x in BL_wFU_thal_subj_list if x.startswith('NOR')])

22

In [207]:
len([x for x in BL_wFU_thal_subj_list if x.startswith('DNO')])

8

In [208]:
len([x for x in BL_wFU_thal_subj_list if x.startswith('UMO')])

11

In [209]:
FU_thal_wBL=FU_thal[FU_thal['subject'].isin(BL_wFU_thal_subj_list)]

In [210]:
FU_thal.shape

(3360, 13)

In [211]:
FU_thal_wBL.shape

(3280, 13)

In [212]:
for df in BL_thal_wFU, FU_thal_wBL:
    df['threshold'] = df.threshold.astype('str')    
    df['threshold'] = df['threshold'].str[-2:]
    df['group'] = df['subject'].str[:3]

/Users/minji/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/minji/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/minji/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

In [213]:
total_connectivity_sum = BL_thal_wFU.groupby(['threshold',
                                          'subject', 
                                          'side']).sum().reset_index()[['subject',
                                                                        'side',
                                                                        'threshold',
                                                                        'total_connectivity_raw']]

total_connectivity_sum.columns = ['subject', 'side', 'threshold', 'total_connectivity_raw_sum']

BL_thal_wFU = pd.merge(BL_thal_wFU,
                  total_connectivity_sum,
                  on=['subject','side','threshold'],
                  how='inner')

In [214]:
BL_thal_wFU['relative_connectivity'] = BL_thal_wFU.total_connectivity_raw / BL_thal_wFU.total_connectivity_raw_sum

In [215]:
FU_total_connectivity_sum = FU_thal_wBL.groupby(['threshold',
                                          'subject', 
                                          'side']).sum().reset_index()[['subject',
                                                                        'side',
                                                                        'threshold',
                                                                        'total_connectivity_raw']]

FU_total_connectivity_sum.columns = ['subject', 'side', 'threshold', 'total_connectivity_raw_sum']

FU_thal_wBL = pd.merge(FU_thal_wBL,
                  FU_total_connectivity_sum,
                  on=['subject','side','threshold'],
                  how='inner')

In [216]:
FU_thal_wBL['relative_connectivity'] = FU_thal_wBL.total_connectivity_raw / FU_thal_wBL.total_connectivity_raw_sum

In [217]:
BL_thal_wFU.shape

(3280, 16)

In [218]:
FU_thal_wBL.shape

(3280, 16)

In [219]:
BL_thal_wFU

,subject,side,cortex,threshold,intra_vol,cortex_volume,thalamus_volume,dti_thalamus_seg_volume,dti_total_connectivity,thalamus_seg_mk,dki_total_connectivity,dki_thalamus_seg_volume,total_connectivity_raw,group,total_connectivity_raw_sum,relative_connectivity
0,DNO22_KMJ,left,SMC,5,1.700106e+06,43622,8897,5533,6456299.0,0.820434,523701.994849,983,6639661.0,DNO,25254904.0,0.262906
1,DNO22_KMJ,left,PC,5,1.700106e+06,63254,8897,3135,3381200.0,0.792782,276989.398803,820,3616246.0,DNO,25254904.0,0.143190
2,DNO22_KMJ,left,OFC,5,1.700106e+06,19165,8897,796,283148.0,0.922246,23009.862315,403,333767.0,DNO,25254904.0,0.013216
3,DNO22_KMJ,left,OCC,5,1.700106e+06,30746,8897,1288,1213218.0,0.912077,98981.008513,430,1345328.0,DNO,25254904.0,0.053270
4,DNO22_KMJ,left,MTC,5,1.700106e+06,19047,8897,6183,4075393.0,0.803132,331544.483088,1099,4210180.0,DNO,25254904.0,0.166707
5,DNO22_KMJ,left,MPFC,5,1.700106e+06,35639,8897,5364,5672801.0,0.768376,459322.756167,922,5872672.0,DNO,25254904.0,0.232536
6,DNO22_KMJ,left,LTC,5,1.700106e+06,56867,8897,6161,1282281.0,0.794816,105624.079410,1168,1367183.0,DNO,25254904.0,0.054135
7,DNO22_KMJ,left,LPFC,5,1.700106e+06,36799,8897,4610,1783724.0,0.804188,145509.511071,894,1869867.0,DNO,25254904.0,0.074040
8,DNO22_KMJ,left,SMC,10,1.700106e+06,43622,8897,4457,6189688.0,0.831583,501659.117050,881,6639661.0,DNO,25254904.0,0.262906
9,DNO22_KMJ,left,PC,10,1.700106e+06,63254,8897,2203,3158551.0,0.814513,258996.074951,660,3616246.0,DNO,25254904.0,0.143190


In [220]:
FU_thal_wBL

,subject,side,cortex,threshold,intra_vol,cortex_volume,thalamus_volume,dti_thalamus_seg_volume,dti_total_connectivity,thalamus_seg_mk,dki_total_connectivity,dki_thalamus_seg_volume,total_connectivity_raw,group,total_connectivity_raw_sum,relative_connectivity
0,DNO22_KMJ,left,SMC,5,1.705381e+06,44376,9467,5954,6339344.0,0.821017,514377.464704,1093,6579919.0,DNO,36902825.0,0.178304
1,DNO22_KMJ,left,PC,5,1.705381e+06,63374,9467,3366,6245838.0,0.857169,507879.787174,743,6541222.0,DNO,36902825.0,0.177255
2,DNO22_KMJ,left,OFC,5,1.705381e+06,18605,9467,1366,902645.0,0.948998,73498.327622,481,1028016.0,DNO,36902825.0,0.027857
3,DNO22_KMJ,left,OCC,5,1.705381e+06,29906,9467,1766,2865466.0,0.871751,235096.766916,444,3139195.0,DNO,36902825.0,0.085067
4,DNO22_KMJ,left,MTC,5,1.705381e+06,18224,9467,6022,10076960.0,0.788035,819225.182138,1041,10417482.0,DNO,36902825.0,0.282295
5,DNO22_KMJ,left,MPFC,5,1.705381e+06,35861,9467,5022,3690194.0,0.749744,301003.140302,945,3852368.0,DNO,36902825.0,0.104392
6,DNO22_KMJ,left,LTC,5,1.705381e+06,57051,9467,6630,1856528.0,0.805417,151901.206845,1164,1943074.0,DNO,36902825.0,0.052654
7,DNO22_KMJ,left,LPFC,5,1.705381e+06,36947,9467,4083,3212237.0,0.826032,262536.910562,824,3401549.0,DNO,36902825.0,0.092176
8,DNO22_KMJ,left,SMC,10,1.705381e+06,44376,9467,4650,6034741.0,0.832376,489777.718224,949,6579919.0,DNO,36902825.0,0.178304
9,DNO22_KMJ,left,PC,10,1.705381e+06,63374,9467,2797,6065122.0,0.881892,493206.024006,650,6541222.0,DNO,36902825.0,0.177255


In [221]:
BL_thal_wFU.groupby(['group','side','cortex','threshold']).count()[['subject']]

subject
group side  cortex threshold         
DNO   left  LPFC   10               8
                   20               8
                   5                8
                   90               8
                   95               8
            LTC    10               8
                   20               8
                   5                8
                   90               8
                   95               8
            MPFC   10               8
                   20               8
                   5                8
                   90               8
                   95               8
            MTC    10               8
                   20               8
                   5                8
                   90               8
                   95               8
            OCC    10               8
                   20               8
                   5                8
                   90               8
                   95               8
            OFC    10               8
                   20               8
                   5                8
                   90               8
                   95               8
...                               ...
UMO   right MPFC   10              11
                   20              11
                   5               11
                   90              11
                   95              11
            MTC    10              11
                   20              11
                   5               11
                   90              11
                   95              11
            OCC    10              11
                   20              11
                   5               11
                   90              11
                   95              11
            OFC    10              11
                   20              11
                   5               11
                   90              11
                   95              11
            PC     10              11
                   20              11
                   5               11
                   90              11
                   95              11
            SMC    10              11
                   20              11
                   5               11
                   90              11
                   95              11

[240 rows x 1 columns]

In [222]:
FU_thal_wBL.groupby(['group','side','cortex','threshold']).count()[['subject']]

subject
group side  cortex threshold         
DNO   left  LPFC   10               8
                   20               8
                   5                8
                   90               8
                   95               8
            LTC    10               8
                   20               8
                   5                8
                   90               8
                   95               8
            MPFC   10               8
                   20               8
                   5                8
                   90               8
                   95               8
            MTC    10               8
                   20               8
                   5                8
                   90               8
                   95               8
            OCC    10               8
                   20               8
                   5                8
                   90               8
                   95               8
            OFC    10               8
                   20               8
                   5                8
                   90               8
                   95               8
...                               ...
UMO   right MPFC   10              11
                   20              11
                   5               11
                   90              11
                   95              11
            MTC    10              11
                   20              11
                   5               11
                   90              11
                   95              11
            OCC    10              11
                   20              11
                   5               11
                   90              11
                   95              11
            OFC    10              11
                   20              11
                   5               11
                   90              11
                   95              11
            PC     10              11
                   20              11
                   5               11
                   90              11
                   95              11
            SMC    10              11
                   20              11
                   5               11
                   90              11
                   95              11

[240 rows x 1 columns]

In [223]:
BL_thal_wFU[BL_thal_wFU.isnull().any(axis=1)]

,subject,side,cortex,threshold,intra_vol,cortex_volume,thalamus_volume,dti_thalamus_seg_volume,dti_total_connectivity,thalamus_seg_mk,dki_total_connectivity,dki_thalamus_seg_volume,total_connectivity_raw,group,total_connectivity_raw_sum,relative_connectivity
1962,NOR24_YJA,right,OFC,5,1.366257e+06,15552,7322,0,0.0,NaN,0.0,0,0.0,NOR,4287972.0,0.0
1965,NOR24_YJA,right,MPFC,5,1.366257e+06,24979,7322,0,0.0,NaN,0.0,0,0.0,NOR,4287972.0,0.0
1970,NOR24_YJA,right,OFC,10,1.366257e+06,15552,7322,0,0.0,NaN,0.0,0,0.0,NOR,4287972.0,0.0
1973,NOR24_YJA,right,MPFC,10,1.366257e+06,24979,7322,0,0.0,NaN,0.0,0,0.0,NOR,4287972.0,0.0
1978,NOR24_YJA,right,OFC,20,1.366257e+06,15552,7322,0,0.0,NaN,0.0,0,0.0,NOR,4287972.0,0.0
1981,NOR24_YJA,right,MPFC,20,1.366257e+06,24979,7322,0,0.0,NaN,0.0,0,0.0,NOR,4287972.0,0.0
1986,NOR24_YJA,right,OFC,90,1.366257e+06,15552,7322,0,0.0,NaN,0.0,0,0.0,NOR,4287972.0,0.0
1989,NOR24_YJA,right,MPFC,90,1.366257e+06,24979,7322,0,0.0,NaN,0.0,0,0.0,NOR,4287972.0,0.0
1994,NOR24_YJA,right,OFC,95,1.366257e+06,15552,7322,0,0.0,NaN,0.0,0,0.0,NOR,4287972.0,0.0
1997,NOR24_YJA,right,MPFC,95,1.366257e+06,24979,7322,0,0.0,NaN,0.0,0,0.0,NOR,4287972.0,0.0


In [224]:
FU_thal_wBL[FU_thal_wBL.isnull().any(axis=1)]

,subject,side,cortex,threshold,intra_vol,cortex_volume,thalamus_volume,dti_thalamus_seg_volume,dti_total_connectivity,thalamus_seg_mk,dki_total_connectivity,dki_thalamus_seg_volume,total_connectivity_raw,group,total_connectivity_raw_sum,relative_connectivity
80,UMO08_LJB,left,SMC,5,1.508875e+06,38707,9000,0,0.0,NaN,0.0,0,0.0,UMO,0.0,NaN
81,UMO08_LJB,left,PC,5,1.508875e+06,65702,9000,0,0.0,NaN,0.0,0,0.0,UMO,0.0,NaN
82,UMO08_LJB,left,OFC,5,1.508875e+06,16897,9000,0,0.0,NaN,0.0,0,0.0,UMO,0.0,NaN
83,UMO08_LJB,left,OCC,5,1.508875e+06,29696,9000,0,0.0,NaN,0.0,0,0.0,UMO,0.0,NaN
84,UMO08_LJB,left,MTC,5,1.508875e+06,14418,9000,0,0.0,NaN,0.0,0,0.0,UMO,0.0,NaN
85,UMO08_LJB,left,MPFC,5,1.508875e+06,27609,9000,0,0.0,NaN,0.0,0,0.0,UMO,0.0,NaN
86,UMO08_LJB,left,LTC,5,1.508875e+06,48651,9000,0,0.0,NaN,0.0,0,0.0,UMO,0.0,NaN
87,UMO08_LJB,left,LPFC,5,1.508875e+06,27973,9000,0,0.0,NaN,0.0,0,0.0,UMO,0.0,NaN
88,UMO08_LJB,left,SMC,10,1.508875e+06,38707,9000,0,0.0,NaN,0.0,0,0.0,UMO,0.0,NaN
89,UMO08_LJB,left,PC,10,1.508875e+06,65702,9000,0,0.0,NaN,0.0,0,0.0,UMO,0.0,NaN


In [225]:
BL_thal_wFU[BL_thal_wFU.dti_thalamus_seg_volume==0]

,subject,side,cortex,threshold,intra_vol,cortex_volume,thalamus_volume,dti_thalamus_seg_volume,dti_total_connectivity,thalamus_seg_mk,dki_total_connectivity,dki_thalamus_seg_volume,total_connectivity_raw,group,total_connectivity_raw_sum,relative_connectivity
1962,NOR24_YJA,right,OFC,5,1.366257e+06,15552,7322,0,0.0,NaN,0.0,0,0.0,NOR,4287972.0,0.0
1965,NOR24_YJA,right,MPFC,5,1.366257e+06,24979,7322,0,0.0,NaN,0.0,0,0.0,NOR,4287972.0,0.0
1970,NOR24_YJA,right,OFC,10,1.366257e+06,15552,7322,0,0.0,NaN,0.0,0,0.0,NOR,4287972.0,0.0
1973,NOR24_YJA,right,MPFC,10,1.366257e+06,24979,7322,0,0.0,NaN,0.0,0,0.0,NOR,4287972.0,0.0
1978,NOR24_YJA,right,OFC,20,1.366257e+06,15552,7322,0,0.0,NaN,0.0,0,0.0,NOR,4287972.0,0.0
1981,NOR24_YJA,right,MPFC,20,1.366257e+06,24979,7322,0,0.0,NaN,0.0,0,0.0,NOR,4287972.0,0.0
1986,NOR24_YJA,right,OFC,90,1.366257e+06,15552,7322,0,0.0,NaN,0.0,0,0.0,NOR,4287972.0,0.0
1989,NOR24_YJA,right,MPFC,90,1.366257e+06,24979,7322,0,0.0,NaN,0.0,0,0.0,NOR,4287972.0,0.0
1994,NOR24_YJA,right,OFC,95,1.366257e+06,15552,7322,0,0.0,NaN,0.0,0,0.0,NOR,4287972.0,0.0
1997,NOR24_YJA,right,MPFC,95,1.366257e+06,24979,7322,0,0.0,NaN,0.0,0,0.0,NOR,4287972.0,0.0


In [226]:
FU_thal_wBL[FU_thal_wBL.dti_thalamus_seg_volume==0]

,subject,side,cortex,threshold,intra_vol,cortex_volume,thalamus_volume,dti_thalamus_seg_volume,dti_total_connectivity,thalamus_seg_mk,dki_total_connectivity,dki_thalamus_seg_volume,total_connectivity_raw,group,total_connectivity_raw_sum,relative_connectivity
80,UMO08_LJB,left,SMC,5,1.508875e+06,38707,9000,0,0.0,NaN,0.0,0,0.0,UMO,0.0,NaN
81,UMO08_LJB,left,PC,5,1.508875e+06,65702,9000,0,0.0,NaN,0.0,0,0.0,UMO,0.0,NaN
82,UMO08_LJB,left,OFC,5,1.508875e+06,16897,9000,0,0.0,NaN,0.0,0,0.0,UMO,0.0,NaN
83,UMO08_LJB,left,OCC,5,1.508875e+06,29696,9000,0,0.0,NaN,0.0,0,0.0,UMO,0.0,NaN
84,UMO08_LJB,left,MTC,5,1.508875e+06,14418,9000,0,0.0,NaN,0.0,0,0.0,UMO,0.0,NaN
85,UMO08_LJB,left,MPFC,5,1.508875e+06,27609,9000,0,0.0,NaN,0.0,0,0.0,UMO,0.0,NaN
86,UMO08_LJB,left,LTC,5,1.508875e+06,48651,9000,0,0.0,NaN,0.0,0,0.0,UMO,0.0,NaN
87,UMO08_LJB,left,LPFC,5,1.508875e+06,27973,9000,0,0.0,NaN,0.0,0,0.0,UMO,0.0,NaN
88,UMO08_LJB,left,SMC,10,1.508875e+06,38707,9000,0,0.0,NaN,0.0,0,0.0,UMO,0.0,NaN
89,UMO08_LJB,left,PC,10,1.508875e+06,65702,9000,0,0.0,NaN,0.0,0,0.0,UMO,0.0,NaN


In [227]:
for df in BL_thal_wFU, FU_thal_wBL:
    df.loc[df.group == 'DNO', 'Group'] = 'OCD'
    df.loc[df.group == 'UMO', 'Group'] = 'OCD'
    df.loc[df.group == 'NOR', 'Group'] = 'HC'

In [228]:
BL_thal_wFU.shape

(3280, 17)

In [229]:
FU_thal_wBL.shape

(3280, 17)

In [230]:
demo = pd.read_csv("/Users/minji/Downloads/ToL_demo.csv")

In [231]:
demo.loc[demo.subject == 'DNO20_JBJ', 'subject'] = 'UMO60_JBJ'

In [232]:
demo.loc[demo.subject == 'UMO18_CSJ', 'subject'] = 'DNO36_CSJ'

In [233]:
BL_thal_wFU_demo = pd.merge(BL_thal_wFU,
                      demo[['subject','age','sex']],
                      left_on='subject',
                      right_on='subject',
                      how='inner')

In [234]:
BL_thal_wFU_demo.shape

(3280, 19)

In [235]:
FU_thal_wBL_demo = pd.merge(FU_thal_wBL,
                      demo[['subject','age','sex']],
                      left_on='subject',
                      right_on='subject',
                      how='inner')

In [236]:
FU_thal_wBL_demo.shape

(3280, 19)

In [237]:
BL_thal_wFU_demo.to_csv("/Users/minji/Desktop/OCD_20190423/BL_thal_wFU_YB.csv")

In [238]:
FU_thal_wBL_demo.to_csv("/Users/minji/Desktop/OCD_20190423/FU_thal_wBL_YB.csv")

In [245]:
BL_thal_wFU_demo = pd.read_csv("/Users/minji/Desktop/OCD_20190423/BL_thal_wFU_YB.csv")

In [246]:
FU_thal_wBL_demo = pd.read_csv("/Users/minji/Desktop/OCD_20190423/FU_thal_wBL_YB.csv")

In [247]:
BL_thal_wFU_demo.shape

(3280, 20)

In [248]:
FU_thal_wBL_demo.shape

(3280, 20)

In [249]:
BL_thal_wFU_demo['time'] = 'BL'

In [250]:
FU_thal_wBL_demo['time'] = 'FU'

In [243]:
#for column in FU_thal_wBL_demo:
    #df['intra_vol'].dtype
    #df['intra_vol'] = df.threshold.astype('int') 

In [251]:
dfs = [BL_thal_wFU_demo, FU_thal_wBL_demo]
final_df = pd.concat(dfs)

In [252]:
final_df.shape

(6560, 21)

In [253]:
final_df

,Unnamed: 0,subject,side,cortex,threshold,intra_vol,cortex_volume,thalamus_volume,dti_thalamus_seg_volume,dti_total_connectivity,...,dki_total_connectivity,dki_thalamus_seg_volume,total_connectivity_raw,group,total_connectivity_raw_sum,relative_connectivity,Group,age,sex,time
0,0,DNO22_KMJ,left,SMC,5,1.700106e+06,43622,8897,5533,6456299.0,...,523701.994849,983,6639661.0,DNO,25254904.0,0.262906,OCD,16,M,BL
1,1,DNO22_KMJ,left,PC,5,1.700106e+06,63254,8897,3135,3381200.0,...,276989.398803,820,3616246.0,DNO,25254904.0,0.143190,OCD,16,M,BL
2,2,DNO22_KMJ,left,OFC,5,1.700106e+06,19165,8897,796,283148.0,...,23009.862315,403,333767.0,DNO,25254904.0,0.013216,OCD,16,M,BL
3,3,DNO22_KMJ,left,OCC,5,1.700106e+06,30746,8897,1288,1213218.0,...,98981.008513,430,1345328.0,DNO,25254904.0,0.053270,OCD,16,M,BL
4,4,DNO22_KMJ,left,MTC,5,1.700106e+06,19047,8897,6183,4075393.0,...,331544.483088,1099,4210180.0,DNO,25254904.0,0.166707,OCD,16,M,BL
5,5,DNO22_KMJ,left,MPFC,5,1.700106e+06,35639,8897,5364,5672801.0,...,459322.756167,922,5872672.0,DNO,25254904.0,0.232536,OCD,16,M,BL
6,6,DNO22_KMJ,left,LTC,5,1.700106e+06,56867,8897,6161,1282281.0,...,105624.079410,1168,1367183.0,DNO,25254904.0,0.054135,OCD,16,M,BL
7,7,DNO22_KMJ,left,LPFC,5,1.700106e+06,36799,8897,4610,1783724.0,...,145509.511071,894,1869867.0,DNO,25254904.0,0.074040,OCD,16,M,BL
8,8,DNO22_KMJ,left,SMC,10,1.700106e+06,43622,8897,4457,6189688.0,...,501659.117050,881,6639661.0,DNO,25254904.0,0.262906,OCD,16,M,BL
9,9,DNO22_KMJ,left,PC,10,1.700106e+06,63254,8897,2203,3158551.0,...,258996.074951,660,3616246.0,DNO,25254904.0,0.143190,OCD,16,M,BL


In [255]:
list(final_df.columns.values)

['Unnamed: 0',
 'subject',
 'side',
 'cortex',
 'threshold',
 'intra_vol',
 'cortex_volume',
 'thalamus_volume',
 'dti_thalamus_seg_volume',
 'dti_total_connectivity',
 'thalamus_seg_mk',
 'dki_total_connectivity',
 'dki_thalamus_seg_volume',
 'total_connectivity_raw',
 'group',
 'total_connectivity_raw_sum',
 'relative_connectivity',
 'Group',
 'age',
 'sex',
 'time']

In [258]:
final_df = final_df.rename(columns={'threshold': 'thr', 'thalamus_volume': 'thalVol', 'cortex_volume': 'cortexVol', 'dti_thalamus_seg_volume': 'T_thalSegVol', 'dki_thalamus_seg_volume': 'K_thalSegVol', 'dti_total_connectivity': 'T_totalConn', 'dki_total_connectivity': 'K_totalConn', 'thalamus_seg_mk': 'thalSeg_mk', 'total_connectivity_raw': 'totalConn_raw', 'total_connectivity_raw_sum': 'totalConn_raw_sum', 'relative_connectivity': 'RelConn' })

In [259]:
list(final_df.columns.values)

['Unnamed: 0',
 'subject',
 'side',
 'cortex',
 'thr',
 'intra_vol',
 'cortexVol',
 'thalVol',
 'T_thalSegVol',
 'T_totalConn',
 'thalSeg_mk',
 'K_totalConn',
 'K_thalSegVol',
 'totalConn_raw',
 'group',
 'totalConn_raw_sum',
 'RelConn',
 'Group',
 'age',
 'sex',
 'time']